In [2]:
import freenect
import cv2
import numpy as np

# --- SETTINGS ---

# 1. Smoothing
# Kernel size for the median blur (must be an odd number)
MEDIAN_BLUR_KERNEL = 25 

# 2. Topography
# The "height" of each elevation step (smaller = more lines/colors)
CONTOUR_STEP = 2
# Colormap for the "land"
LAND_COLORMAP = cv2.COLORMAP_JET # (blue-green-yellow-red)

# 3. Water
# Any pixel with a depth value *below* this will be water.
# Tune this value (0-255) to match your sandbox's base level.
SEA_LEVEL = 100
# Color of the water (in BGR format)
WATER_COLOR = [255, 0, 0] # Bright Blue

# --- KINECT FUNCTIONS ---

def get_depth():
    """
    Gets 8-bit normalized (0-255) depth data from the Kinect.
    Invalid pixels (occlusions) are set to 255.
    """
    array, _ = freenect.sync_get_depth() 
    np.clip(array, 0, 1023, out=array)
    array >>= 2
    array = array.astype(np.uint8)
    return array

def get_video():
    """
    Gets 8-bit BGR (OpenCV-ready) video data from the Kinect.
    """
    array, _ = freenect.sync_get_video()
    array = cv2.cvtColor(array, cv2.COLOR_RGB2BGR) 
    return array

# --- MAIN PROGRAM ---

if __name__ == "__main__":
    while True:
        # --- 1. ACQUISITION ---
        # Get raw depth for masking and RGB for display
        depth_raw = get_depth()
        rgb = get_video()
        
        # --- 2. SMOOTHING ---
        # Apply a median blur to remove "salt-and-pepper" noise
        depth_smooth = cv2.medianBlur(depth_raw, MEDIAN_BLUR_KERNEL)
        
        # --- 3. ANALYSIS & VISUALIZATION ---
        
        # A. Create discrete elevation "steps"
        quantized_depth = (depth_smooth // CONTOUR_STEP) * CONTOUR_STEP
        
        # B. Apply colormap to the quantized "land"
        depth_colormap = cv2.applyColorMap(quantized_depth, LAND_COLORMAP)
        
        # C. Find contour lines (the edges of the "steps")
        edges = cv2.Canny(quantized_depth, 100, 200)

        # D. Draw contours (black lines) onto the colormap
        depth_colormap[edges == 255] = [0, 0, 0] 
        
        # E. Add "water" by coloring all pixels below sea level
        # We use the smoothed depth map for this check
        depth_colormap[depth_smooth < SEA_LEVEL] = WATER_COLOR
        
        # --- 4. MASKING ---
        
        # Find all "invalid" pixels (occlusion shadows)
        # (These are pixels where raw depth was 2047, which became 255)
        invalid_mask = (depth_raw == 255)
        
        # Apply the mask to the final image
        depth_colormap[invalid_mask] = [0, 0, 0]
        
        # --- 5. DISPLAY ---
        
        # Show the debug RGB stream
        cv2.imshow('Kinect RGB (Debug)', rgb)
        
        # Show the final, processed AR Sandbox output
        cv2.imshow('AR Sandbox Output', depth_colormap) 
        
        # Wait for 1ms and check if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources and destroy windows
    cv2.destroyAllWindows()

[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Lost too many packets, resyncing...
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Lost too many packets, resyncing...
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Invalid magic ffff
[Stream 70] Lost too many packets, resyncing...
[Stream 70] Invalid

In [3]:
from sandbox.sensor import kinectv1

FileNotFoundError: [Errno 2] No such file or directory: '/home/geologydept/Desktop/GeoBox/notebooks/tutorials/06_LoadSaveTopoModule/saved_DEMs/'